In [ ]:
#Gabi Bar - 066466061
#Pavel Kaganovich - 304472707
#s3 version https://hwbigdata2020.s3-eu-west-1.amazonaws.com/hw3/hw3_gabi+bar_066466061_Pavel+Kaganovich_304472707.ipynb

In [11]:
from bson.json_util import loads
import boto3
from decimal import Decimal
import bson
from boto3.dynamodb.conditions import Key, Attr

In [25]:
date_format = '"%Y-%m-%d, %H:%M:%S"'
table_name = 'nyt2_3'

In [6]:
items = []
with open('nyt2.json', 'r') as file1:
    while True:
        line = file1.readline()
        if not line:
            break
            
        d = loads(line,  parse_float=Decimal)
        d['_id'] = str(d['_id'])
        d['title_id'] = '{}_{}'.format(d['title'], str(d['_id'])) 
        d['bestsellers_date'] = d['bestsellers_date'].strftime(date_format)
        d['published_date'] = d['published_date'].strftime(date_format)
        d['price'] = Decimal(str(10.3))
        items.append(d)
len(items)

10195

In [84]:
dynamodb = boto3.resource('dynamodb', region_name='eu-west-1')

client = boto3.client('dynamodb', region_name='eu-west-1')

In [8]:
table = dynamodb.create_table(
    TableName=table_name,
    KeySchema=[
        {
            'AttributeName': 'author',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'title_id',
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'author',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'title_id',
            'AttributeType': 'S'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 30
    }
)


In [9]:
response = client.list_tables(
    ExclusiveStartTableName='string',
    Limit=7
)
response

{'TableNames': [],
 'ResponseMetadata': {'RequestId': 'MKS95I8V6MIQNL9TFPVQTD7307VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 28 May 2020 11:45:12 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '17',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'MKS95I8V6MIQNL9TFPVQTD7307VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1315925753'},
  'RetryAttempts': 0}}

In [10]:
table = dynamodb.Table('nyt2_3')
print(table.creation_date_time)
with table.batch_writer() as batch:
    for index, r in enumerate(items):
        batch.put_item(Item=r)
        if index % 1000 == 0:
            print(index)

2020-05-28 14:45:02.150000+03:00
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [14]:
def get_author_books(author, projection=None, filterExpression = None, expressionAttributeNames = None):
    args = {'KeyConditionExpression': Key('author').eq(author)}
    if projection is not None:
        args['ProjectionExpression'] = projection
    
    if filterExpression:
        args['FilterExpression'] = filterExpression
    if expressionAttributeNames is not None:
        args['ExpressionAttributeNames'] = expressionAttributeNames
    response = table.query(
        **args
    )
    
    return response['Items']

#d

In [16]:
books = get_author_books('John Sandford')
books

[{'bestsellers_date': '"2010-09-26, 00:00:00"',
  'title_id': 'BAD BLOOD_5b4aa4ead3089013507dbb16',
  'rank': Decimal('4'),
  'weeks_on_list': Decimal('1'),
  'publisher': 'Putnam',
  '_id': '5b4aa4ead3089013507dbb16',
  'rank_last_week': Decimal('0'),
  'description': 'Virgil Flowers is summoned to investigate a monstrous multigenerational conspiracy.',
  'price': Decimal('10.3'),
  'published_date': '"2010-10-10, 00:00:00"',
  'author': 'John Sandford',
  'title': 'BAD BLOOD',
  'amazon_product_url': 'http://www.amazon.com/Bad-Blood-Virgil-Flowers-novel/dp/0399156909?tag=NYTBS-20'},
 {'bestsellers_date': '"2010-10-03, 00:00:00"',
  'title_id': 'BAD BLOOD_5b4aa4ead3089013507dbb2f',
  'rank': Decimal('9'),
  'weeks_on_list': Decimal('2'),
  'publisher': 'Putnam',
  '_id': '5b4aa4ead3089013507dbb2f',
  'rank_last_week': Decimal('4'),
  'description': 'Virgil Flowers is summoned to investigate a monstrous multigenerational conspiracy.',
  'price': Decimal('10.3'),
  'published_date': '"2

#e

In [17]:
books = get_author_books('John Sandford', "title,author,publication_date")
books

[{'author': 'John Sandford', 'title': 'BAD BLOOD'},
 {'author': 'John Sandford', 'title': 'BAD BLOOD'},
 {'author': 'John Sandford', 'title': 'BAD BLOOD'},
 {'author': 'John Sandford', 'title': 'BAD BLOOD'},
 {'author': 'John Sandford', 'title': 'BURIED PREY'},
 {'author': 'John Sandford', 'title': 'BURIED PREY'},
 {'author': 'John Sandford', 'title': 'BURIED PREY'},
 {'author': 'John Sandford', 'title': 'BURIED PREY'},
 {'author': 'John Sandford', 'title': 'BURIED PREY'},
 {'author': 'John Sandford', 'title': 'BURIED PREY'},
 {'author': 'John Sandford', 'title': 'BURIED PREY'},
 {'author': 'John Sandford', 'title': 'DEADLINE'},
 {'author': 'John Sandford', 'title': 'DEADLINE'},
 {'author': 'John Sandford', 'title': 'DEADLINE'},
 {'author': 'John Sandford', 'title': 'DEADLINE'},
 {'author': 'John Sandford', 'title': 'DEADLINE'},
 {'author': 'John Sandford', 'title': 'DEADLINE'},
 {'author': 'John Sandford', 'title': 'DEEP FREEZE'},
 {'author': 'John Sandford', 'title': 'DEEP FREEZE'},


#f

In [19]:
books = get_author_books('Alex Berenson', 'title,price,#rank', Attr("title").begins_with('THE'),{'#rank':'rank'})
books

[{'price': Decimal('10.3'),
  'rank': Decimal('8'),
  'title': 'THE COUNTERFEIT AGENT'},
 {'price': Decimal('10.3'),
  'rank': Decimal('13'),
  'title': 'THE COUNTERFEIT AGENT'},
 {'price': Decimal('10.3'),
  'rank': Decimal('19'),
  'title': 'THE COUNTERFEIT AGENT'},
 {'price': Decimal('10.3'),
  'rank': Decimal('9'),
  'title': 'THE MIDNIGHT HOUSE'},
 {'price': Decimal('10.3'),
  'rank': Decimal('12'),
  'title': 'THE MIDNIGHT HOUSE'},
 {'price': Decimal('10.3'),
  'rank': Decimal('19'),
  'title': 'THE MIDNIGHT HOUSE'},
 {'price': Decimal('10.3'),
  'rank': Decimal('10'),
  'title': 'THE NIGHT RANGER'},
 {'price': Decimal('10.3'), 'rank': Decimal('8'), 'title': 'THE PRISONER'},
 {'price': Decimal('10.3'),
  'rank': Decimal('6'),
  'title': 'THE SECRET SOLDIER'},
 {'price': Decimal('10.3'),
  'rank': Decimal('9'),
  'title': 'THE SECRET SOLDIER'},
 {'price': Decimal('10.3'),
  'rank': Decimal('19'),
  'title': 'THE SECRET SOLDIER'},
 {'price': Decimal('10.3'),
  'rank': Decimal('8'),

#g

In [21]:
response = table.scan(
    FilterExpression=Attr("title").begins_with('THE')
    )

response['ScannedCount'], response['Count']

(2238, 777)

In [23]:
#h

In [56]:
%%time
response = table.scan(
    FilterExpression = Attr("weeks_on_list").eq(0) & Attr('rank').lt(16),
    ReturnConsumedCapacity='INDEXES'
    )

response['ScannedCount'], response['Count'], response['ConsumedCapacity'], len(response['Items'])

CPU times: user 25.7 ms, sys: 8.08 ms, total: 33.8 ms
Wall time: 726 ms


(2238,
 0,
 {'TableName': 'nyt2_3',
  'CapacityUnits': 128.5,
  'Table': {'CapacityUnits': 128.5}},
 0)

#I

In [26]:
response = client.update_table(
    TableName = table_name,
    AttributeDefinitions=[
                    {
                        'AttributeName': 'weeks_on_list',
                        'AttributeType': 'N'
                    }],
    GlobalSecondaryIndexUpdates=[
        {
            'Create': {
                'IndexName': 'weeks_on_list_gsi',
                'KeySchema': [
                    {
                        'AttributeName': 'weeks_on_list',
                        'KeyType': 'HASH'
                    },
                    
                ],
                'Projection': {
                    'ProjectionType': 'ALL'
                },
                'ProvisionedThroughput': {
                    'ReadCapacityUnits': 10,
                    'WriteCapacityUnits': 10
                }
            }
        }
    ]
)

#j

In [59]:
%%time

response = table.query(
    IndexName='weeks_on_list_gsi',
    KeyConditionExpression = Key('weeks_on_list').eq(0),
    FilterExpression= Attr('#rank').lt(16),
    ReturnConsumedCapacity='INDEXES',
        ExpressionAttributeNames={'#rank': 'rank'},
    ProjectionExpression='#rank'
)

response['ScannedCount'], response['Count'], response['ConsumedCapacity']

CPU times: user 4.78 ms, sys: 1.4 ms, total: 6.18 ms
Wall time: 148 ms


(1965,
 0,
 {'TableName': 'nyt2_3',
  'CapacityUnits': 111.5,
  'Table': {'CapacityUnits': 0.0},
  'GlobalSecondaryIndexes': {'weeks_on_list_gsi': {'CapacityUnits': 111.5}}})

In [ ]:
#k

In [76]:
books = get_author_books('Elizabeth George', 'title_id', Attr("title").eq('BELIEVING THE LIE'))
for book in books:
    response = table.update_item(
        Key={
            'author': 'Elizabeth George',
             'title_id': book['title_id']
        },
        UpdateExpression="set price = :p",
        ExpressionAttributeValues={
            ':p': 15,
        },
         ReturnValues="NONE"
        )

In [77]:
#l

In [80]:
books = get_author_books('Elizabeth George', 'title_id', Attr("title").eq('BELIEVING THE LIE'))

for book in books:
    table.update_item(
        Key={
            'author': 'Elizabeth George',
            'title_id': book['title_id'],
        },
        UpdateExpression='remove publisher',
        ReturnValues="NONE"
    )

In [83]:
#m

In [82]:
books = get_author_books('Elizabeth George', 'title_id', Attr("title").eq('THIS BODY OF DEATH'))

for book in books:
    response = table.delete_item(
        Key={
            'author': 'Elizabeth George',
            'title_id': book['title_id'],
        },
        ReturnValues="NONE"
    )